# Modelo NK

1960 - Kauffman

- $N$: Representa el número de componentes o genes en el sistema (por ejemplo, genes en un genoma o decisiones en una organización).

- $K$: Es el grado de interdependencia entre los componentes. Cada componente depende de $K$ otros componentes además de sí mismo.

Por ejemplo: $N= 3$, tal que: 

- ¿Invertir en I+D? (0: no, 1: sí)

- ¿Vender online? (0: no, 1: sí)

- ¿Producir localmente? (0: no, 1: sí)

Y tenemos todas las configuraciones:

In [2]:
from itertools import product

N = 3

local_states = list(product([0, 1], repeat=N))
for c in local_states:
    print(c)

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


Asimismo $K$ determina de que otra variable depende. En nuestro caso puede suceder que $K = 1$ o $K = 2$. Que dependa únicamente de otra variable, como que dependa de todas las variables. La relación de dependencia no esta determinada, pero podría ser el vecino $0$ depende de $1$, $1$ de $2$ y $2$ de $3$. Entre más dependencias.

In [3]:
import random

K = 1

dependencias = {}

for i in range(N):
    otros = list(range(N))
    otros.remove(i)
    seleccionados = random.sample(otros, K)
    dependencias[i] = [i] + seleccionados

print(dependencias)

{0: [0, 1], 1: [1, 0], 2: [2, 1]}


De esta forma, entonces cada componente $i$ tiene una función local de fitness $f_i$ que depende del valor propio del componente y de los valores de los $K$ componentes de los que depende.

Por ejemplo: Si la función local de fitness (rentabilidad en nuestro caso) para el componente $0$ depende únicamente de "Invertir en I+D" ($0$) y "Vender online" ($1$), ($K = 1$), entonces:

Tal que la rentabilidad (fitness local) para "Invertir en I+D" podría ser menor cuando se invierte $(1)$ pero no se vende online $(0)$. Entonces, las configuraciones globales que tienen $(1,0,1)$ o $(1,0,0)$ coinciden en esos primeros dos bits y comparten ese fitness local.



In [4]:
import numpy as np 

fitness_local = {}
for i in range(N):
    estados_i = list(product([0, 1], repeat= K + 1))
    fitness_local[i] = {}
    for estado in estados_i:
        fitness_local[i][estado] = np.random.rand()

def calcular_fitness_local(i, global_state):
    subestado = tuple(global_state[j] for j in dependencias[i])
    return fitness_local[i][subestado]

fitness_local_state = {}
for i in range(N):
    fitness_local_state[i] = {}
    for state in product([0, 1], repeat=N):
        fitness_local_state[i][state] = calcular_fitness_local(i, state)



In [5]:
fitness_local

{0: {(0, 0): 0.7956704876031627,
  (0, 1): 0.3897582834669465,
  (1, 0): 0.32716243282049573,
  (1, 1): 0.45584467515591676},
 1: {(0, 0): 0.9334623350515373,
  (0, 1): 0.834458676350748,
  (1, 0): 0.010086772069321737,
  (1, 1): 0.07880057227285375},
 2: {(0, 0): 0.5472825658280484,
  (0, 1): 0.4034754887948723,
  (1, 0): 0.3774559937364208,
  (1, 1): 0.803716252079826}}

In [6]:
fitness_local_state

{0: {(0, 0, 0): 0.7956704876031627,
  (0, 0, 1): 0.7956704876031627,
  (0, 1, 0): 0.3897582834669465,
  (0, 1, 1): 0.3897582834669465,
  (1, 0, 0): 0.32716243282049573,
  (1, 0, 1): 0.32716243282049573,
  (1, 1, 0): 0.45584467515591676,
  (1, 1, 1): 0.45584467515591676},
 1: {(0, 0, 0): 0.9334623350515373,
  (0, 0, 1): 0.9334623350515373,
  (0, 1, 0): 0.010086772069321737,
  (0, 1, 1): 0.010086772069321737,
  (1, 0, 0): 0.834458676350748,
  (1, 0, 1): 0.834458676350748,
  (1, 1, 0): 0.07880057227285375,
  (1, 1, 1): 0.07880057227285375},
 2: {(0, 0, 0): 0.5472825658280484,
  (0, 0, 1): 0.3774559937364208,
  (0, 1, 0): 0.4034754887948723,
  (0, 1, 1): 0.803716252079826,
  (1, 0, 0): 0.5472825658280484,
  (1, 0, 1): 0.3774559937364208,
  (1, 1, 0): 0.4034754887948723,
  (1, 1, 1): 0.803716252079826}}

El fitness total o fitness global del sistema se define como el promedio de los fitness locales:

$$ F(x) = \frac{1}{N}\sum_{i=0}^{N-1} f_i(x_{i},\mathbf x_{k_i})$$

donde $\mathbf x_{k_i}$ indica los estados de las variables que son dependientes de $i$.

In [5]:
global_fitness = {}
for state in local_states: 
    global_fitness[state] = sum(fitness_local_state[i][state] for i in range(N))/N

In [6]:
global_fitness

{(0, 0, 0): 0.36006935193488204,
 (0, 0, 1): 0.5124681024683158,
 (0, 1, 0): 0.22444268414242363,
 (0, 1, 1): 0.35117501397602213,
 (1, 0, 0): 0.2658120658769341,
 (1, 0, 1): 0.263628905519374,
 (1, 1, 0): 0.30733990264858024,
 (1, 1, 1): 0.279490321591185}

In [258]:
import random
import numpy as np
from itertools import product

class Landscape:
    def __init__(self, N=3, K=1):
        self.N = N                                                            # Number of attributes
        self.K = K                                                            # Number of complementaries
        self.local_states = list(product([0, 1], repeat=N))                   # Generate the space of states [0,1]^N
        self.dependencias = self._generar_dependencias()                      # Generates dependencies between attributes according to K 
        self.fitness_local = self._inicializar_fitness_local()                # Assigns the random fitness to each complementarity
        self.fitness_local_state = self._calcular_fitness_local_state()       # Maps each space of state to the specific case of complementarity
        self.global_fitness = self._calcular_global_fitness()                 # Generates the landscape 

    def _generar_dependencias(self):
        dependencias = {}
        for i in range(self.N):
            otros = list(range(self.N))                                       # List all possible attributes
            otros.remove(i)                                                   # Removes itself, because K tells abut the cmplementarity with the rest of attributes
            seleccionados = random.sample(otros, self.K)                      # Randomly choose the complementarities 
            dependencias[i] = [i] + seleccionados                             # Creates a list of the complementarities and adds that it depends on itself (This is for easier computation) 
        return dependencias

    def _inicializar_fitness_local(self):
        fitness_local = {}
        for i in range(self.N):                                                   # List all possible attributes
            estados_i = list(product([0, 1], repeat=len(self.dependencias[i])))   # Assigns a fitness to each complementarity
            fitness_local[i] = {estado: np.random.rand() for estado in estados_i} # The fitness is random
        return fitness_local

    def calcular_fitness_local(self, i, global_state):
        subestado = tuple(global_state[j] for j in self.dependencias[i])      # To a given state finds it's corresponding complementary generates a "substate"
        return self.fitness_local[i][subestado]

    def _calcular_fitness_local_state(self):
        fitness_local_state = {}
        for i in range(self.N):
            fitness_local_state[i] = {}
            for state in self.local_states:
                fitness_local_state[i][state] = self.calcular_fitness_local(i, state) # Maps each state with its corresponding substate
        return fitness_local_state

    def _calcular_global_fitness(self):
        global_fitness = {} 
        for state in self.local_states:
            global_fitness[state] = sum(self.fitness_local_state[i][state] for i in range(self.N)) / self.N # Calculates the mean of the fitness of each attribute according to the complementarities "available" in that state
        return global_fitness

    # Functions to show the process if wanted

    # Show states
    def mostrar_estados(self):
        print("All possible states:") 
        for state in self.local_states:
            print(state)
    #  Show dependencies
    def mostrar_dependencias(self):
        print("\nDependencies of each state:")
        for i, deps in self.dependencias.items():
            print(f"Nodo {i}: {deps}")

    # Show local fitness (of each attribute)
    def mostrar_fitness_local(self, nodo=None, show = False):
        print("\nLocal Fitness:")
        if len(self.local_states) > 10 and show == False:
            print(f"\nWarning: There are {len(self.local_states)} states. Use `show=True` to show them all.")
        else:    
            if nodo is not None:
                print(f"Node {nodo}:")
                for estado, fit in self.fitness_local_state[0].items():
                    print(f"  State {estado}: {fit:.4f}")
            else:
                for i in range(self.N):
                    print(f"Node {i}:")
                    for estado, fit in self.fitness_local_state[i].items():
                        print(f"  State {estado}: {fit:.4f}")

    # Show global fitness (of each state)
    def mostrar_fitness_global(self, num_estados=5):
        print("\nGlobal Fitness (firsts", num_estados, "states):")
        for i, state in enumerate(self.local_states):
            if i >= num_estados:
                break
            print(f"  State {state}: {self.global_fitness[state]:.4f}")

    # Show the state with maximum fitness (just for comparison to see if firm are able to arrive to that state.
    def encontrar_optimo_global(self):
        mejor_estado = max(self.global_fitness.items(), key=lambda x: x[1])
        return mejor_estado[0], mejor_estado[1]                               # Gives us the state and the fitness 

class Firm:
    def __init__(self, landscape, estadoinicial = None, time = 0):

        # Characteristic Environment
        self.landscape = landscape
        self.N = landscape.N                                                  # Number of attributes
        self.K = landscape.K                                                  # Number of complementaries
        self.local_states = landscape.local_states                            # Space of states [0,1]^N
        self.global_fitness = landscape.global_fitness                        # Landscape
        
        # Characteristic Firm
        if estadoinicial == None:
            self.estado_actual = random.choice(self.local_states)             # Attributes an initial state if not started
        else:
            self.estado_actual = estadoinicial
            
        self.fitness_actual = self.global_fitness[self.estado_actual]         # Maps the state to the fitness
        self.timeborn = time
        self.historial = [{'state': self.estado_actual, 
                           'fitness': self.fitness_actual,
                            'type of search': -1}]                   # Creates a historial
                    
    def _obtener_vecinos(self):
        """Give us the neighbors (just chainging one bit from the actual state)."""
        vecinos = []
        for i in range(self.N):
            vecino = list(self.estado_actual)                                 # The neigbor is equal to the state (except for one bit)
            vecino[i] = 1 - vecino[i]                                         # Flips one bit at the time 0->1 o 1->0
            vecinos.append(tuple(vecino))
        return vecinos

    def adaptacion(self, BL, GS, verbose= True):
        if verbose:
            print(f"\nStarting search from the state: {self.estado_actual} (fitness={self.fitness_actual:.4f})")

        candidatos = [(self.estado_actual, self.fitness_actual)]

        # Results from local adaptation
        estado_BL, fitness_BL = self.simular_busqueda_local()
        if BL == False:
            fitness_BL = 0
        candidatos.append((estado_BL, fitness_BL))
        
        # Results from long jump
        # Implicitly, the effectiveness of a long jump is going to depend on the fitness (negatively)
        estado_GS, fitness_GS = self.grandes_saltos()
        if GS == False:
            fitness_GS = 0
        candidatos.append((estado_GS, fitness_GS))

        # Chooses the best research outcome, or none.
        mejor_indice, (estado_nuevo, fitness_nuevo) = max(enumerate(candidatos), 
                                                          key=lambda x: x[1][1] 
                                                        )
        self.estado_actual = estado_nuevo
        self.fitness_actual = fitness_nuevo
            
        if verbose:
            if mejor_indice == 0:
                print(f"No Mutation: Stays in {self.estado_actual} (fitness={self.fitness_actual:.4f})")
            elif  mejor_indice == 1:
                print(f"Local Adaptation: Moves to {self.estado_actual} (fitness={self.fitness_actual:.4f})")
            elif mejor_indice == 2: 
                print(f"Long-Jump: Moves to {self.estado_actual} (fitness={self.fitness_actual:.4f})") 
            
        self.historial.append({'state': self.estado_actual, 
                               'fitness': self.fitness_actual,
                               'type of search': mejor_indice})


    def simular_busqueda_local(self):
        vecinos = self._obtener_vecinos()                                   # Collects neighbors
        vecinos_mejores = [(v, self.global_fitness[v]) for v in vecinos     # Calculate the fitness of the neigbors and determines which are greater than actual state
                           if self.global_fitness[v] > self.fitness_actual]
        if vecinos_mejores:
            estado_actualBL, fitness_actualBL = random.choice(vecinos_mejores)    # Random choice of improvement among neighbors
        else:
            estado_actualBL, fitness_actualBL = self.estado_actual, 0       # Local optimum

        return estado_actualBL, fitness_actualBL

    def grandes_saltos(self):
        estado_actualGS = random.choice(self.local_states)                  # Random choice of improvement among all states
        fitness_actualGS = self.global_fitness[estado_actualGS]     
        return estado_actualGS, fitness_actualGS


    def mostrar_historial(self, historial):
        """Shows the historial of search."""
        print("\nSearching historial:")
        for paso in historial:
            if paso['type of search'] == 1:
                print(f"Local Adaptation: State: {paso['state']} -> Fitness: {paso['fitness']:.4f}")
            elif paso['type of search'] == 2:
                print(f"Long-Jump: State: {paso['state']} -> Fitness: {paso['fitness']:.4f}")
            elif paso['type of search'] == -1:
                print(f"Initial State: State: {paso['state']} -> Fitness: {paso['fitness']:.4f}")
            elif paso['type of search'] == 0:
                print(f"No Mutation: State: {paso['state']} -> Fitness: {paso['fitness']:.4f}")
            else:
                print(f"Check errors!")

class Industry:
    def __init__(self, n_firms=5, N=3, K=1, BL = True, GS = True, Exit = True):
        self.n_firms = n_firms
        self.BL = BL
        self.GS = GS
        self.Exit = Exit
        self.landscape = Landscape(N=N, K=K)                             # The Industry generates the landscape
        self.firms = [Firm(self.landscape) for _ in range(n_firms)]      # N firms born
        self.local_states = self.landscape.local_states
        self.n_newfirms  = 0
        self.history_industry  = []
        self.history_forms = []
        self.record_history(0)
        
    def adapt_all(self, verbose=False):
        for f in self.firms:
            f.adaptacion(BL = self.BL, GS = self.GS, verbose=verbose)

    def exit_and_entry(self, t):
        """Aplica salida y entrada de firmas según fitness relativa y carga genética."""
        if not self.firms:
            return

        # ---------- 1. Exit ----------
        max_fitness = max(f.fitness_actual for f in self.firms)
        supervivientes = []
        for f in self.firms:
            fitness_relativa = f.fitness_actual / max_fitness if max_fitness > 0 else 0  # Calculates likelihood of dying
            if random.random() < fitness_relativa:                                       # Determines if the firm dies or not
                supervivientes.append(f)

        self.firms = supervivientes

        # ---------- 2. Entrada ----------
        n_new = self.n_firms - len(self.firms)
        self.n_newfirms = n_new
        
        if n_new <= 0:
            return

        mean_fitness = np.mean([f.fitness_actual for f in self.firms]) if self.firms else 0
        prob_copia = 1 - (mean_fitness / max_fitness) if max_fitness > 0 else 0          # Genetic Load

        new_firms = []
        for _ in range(n_new):
            if self.firms and random.random() < prob_copia:
                # Choosing a random firm according to its fitness 
                fitnesses = np.array([f.fitness_actual for f in self.firms])
                probs = fitnesses / fitnesses.sum() if fitnesses.sum() > 0 else np.ones(len(self.firms)) / len(self.firms)
                idx = np.random.choice(len(self.firms), p=probs)
                f_to_copy = self.firms[idx]
                estadoinicial = f_to_copy.estado_actual
            else:
                # Random state
                estadoinicial = random.choice(self.local_states)
            
            f_new = Firm(self.landscape, estadoinicial = estadoinicial, time = t+1)
            new_firms.append(f_new)

        self.firms.extend(new_firms)
        
    def record_history(self, t):
        if not self.firms:
            return
    
        fitnesses = np.array([f.fitness_actual for f in self.firms])
        # --- Industry aggregated snapshot ---
        snapshot_industry = {
            'time': t,
            'num_firms': len(self.firms),
            'max_fitness': np.max(fitnesses),
            'min_fitness': np.min(fitnesses),
            'avg_fitness': np.mean(fitnesses),
            'num_entries': float(self.n_newfirms),
            'num_org_forms': len(set([tuple(f.estado_actual) for f in self.firms]))
        }
        self.history_industry.append(snapshot_industry)
    
        # --- Distribution of organizational forms ---
        form_dict = {}
        for f in self.firms:
            s = tuple(f.estado_actual)
            if s not in form_dict:
                form_dict[s] = []
            form_dict[s].append(f.fitness_actual)
    
        bins = np.arange(0, 1.05, 0.05)
        counts, _ = np.histogram(fitnesses, bins=bins)
    
        self.history_forms.append({
            'time': t,
            'bins': bins.tolist(),
            'counts': counts.tolist()
        })
        
    def show_firms(self):
        print(f"\n--- Estado de la industria en t={self.time} ---")
        for i, f in enumerate(self.firms):
            print(f"Firm {i}: State {f.estado_actual}, Fitness {f.fitness_actual:.4f}")

    def step(self, t):
        self.adapt_all()
        if self.Exit == True:
            self.exit_and_entry(t + 1)
        self.record_history(t + 1)

import numpy as np

class IndustrySimulation:
    def __init__(self, M=100, T=100, **industry_params):
        self.M = M
        self.T = T
        self.industry_params = industry_params

        # Crear M industrias con los mismos parámetros
        self.industries = [Industry(**self.industry_params) for _ in range(M)]

        # Historial agregado
        self.history_industry_avg = []
        self.history_forms_avg = []

    def run(self):
        self.update_history(0)
        for t in range(self.T):
            # Simular un paso en cada industria
            for ind in self.industries:
                ind.step(t)

            # Actualizar históricos
            self.update_history(t + 1)

    def update_history(self, t):
        # ---- Agregación del history_industry ----
        valid_snaps = [ind.history_industry[-1] for ind in self.industries if ind.history_industry]
        if valid_snaps:
            self.history_industry_avg.append({
                'time': t,
                'avg_max_fitness': np.mean([s['max_fitness'] for s in valid_snaps]),
                'avg_min_fitness': np.mean([s['min_fitness'] for s in valid_snaps]),
                'avg_avg_fitness': np.mean([s['avg_fitness'] for s in valid_snaps]),
                'avg_num_entries': np.mean([s['num_entries'] for s in valid_snaps]),
                'avg_num_org_forms': np.mean([s['num_org_forms'] for s in valid_snaps]),
                'avg_num_firms': np.mean([s['num_firms'] for s in valid_snaps])
            })

        # ---- Agregación del history_forms ----
        hist_list = [i.history_forms[-1]['counts'] for i in self.industries if i.history_forms]
        if hist_list:
            bins = self.industries[0].history_forms[-1]['bins']
            avg_counts = np.mean(hist_list, axis=0)
            self.history_forms_avg.append({
                'time': t,
                'bins': bins,
                'avg_counts': avg_counts.tolist()
            })


In [255]:
if __name__ == "__main__":
    sim1 = IndustrySimulation(
        M=100,
        T=50,
        n_firms=100,
        N=10,
        K=3,
        BL=True,
        GS=False,
        Exit=False
    )
    

    sim1.run()


In [256]:
sim1.industry_params

{'n_firms': 100, 'N': 10, 'K': 3, 'BL': True, 'GS': False, 'Exit': False}

In [257]:
sim1.history_industry_avg

[{'time': 0,
  'avg_max_fitness': np.float64(0.7158465171815692),
  'avg_min_fitness': np.float64(0.2848148491041943),
  'avg_avg_fitness': np.float64(0.5001054474125326),
  'avg_num_entries': np.float64(0.0),
  'avg_num_org_forms': np.float64(95.01),
  'avg_num_firms': np.float64(100.0)},
 {'time': 1,
  'avg_max_fitness': np.float64(0.752346730220871),
  'avg_min_fitness': np.float64(0.34342032438105674),
  'avg_avg_fitness': np.float64(0.5603528092477272),
  'avg_num_entries': np.float64(0.0),
  'avg_num_org_forms': np.float64(91.98),
  'avg_num_firms': np.float64(100.0)},
 {'time': 2,
  'avg_max_fitness': np.float64(0.7665052171941058),
  'avg_min_fitness': np.float64(0.40006060725896886),
  'avg_avg_fitness': np.float64(0.6085922608180023),
  'avg_num_entries': np.float64(0.0),
  'avg_num_org_forms': np.float64(81.21),
  'avg_num_firms': np.float64(100.0)},
 {'time': 3,
  'avg_max_fitness': np.float64(0.7674443145039292),
  'avg_min_fitness': np.float64(0.4455891743933287),
  'avg_